# Interfacing with the directRF addon board

The directrf is an addon board to the ZCU216 to convert the raw ADC and DAC signals into a format suitable for qubit instrumentation.
For this purpose, the addon board contains analog filters, BalUns and can switch between different nyquist zones.

In [1]:
import qiclib as ql
from qiclib.code import *

ip = "192.168.99.206"

qic = ql.QiController(ip)

[QiController] qiclib version: 1.0.0 +29 (ea3f939c, 15.05.2024 17:06) <dirty>
[QiController] Establishing remote connection to 192.168.99.206...
[QiController] Detected QiController running on ZCU216 board
[QiController] Firmware build time: 13.05.2024 09:46:23 (Revision CA4FAB10)
[QiController] Firmware with 6 digital unit cells detected.


## APIs in qiclib

`qiclib` provides two APIs to interface with the board:
- The `qiclib.hardware.direct_rf` provides abstractions for the entire RFdc + frontend board chain. This is the easiest and also preferred way of interfacing with the directRF.
- The `qiclib.hardware.direct_rf_addon` provides low-level control and enables manually setting the different configuration options. This can be used together with the `qiclib.hardware.rfdc` module to mimic the methods provided by the `direct_rf` API.

Whenever possible, the former method should be used. Wrong manual usage may reduce the lifespan of the analog board for certain configurations.

In [2]:
# Using the direct_rf API to set a frequency of 1 GHz at the the output
qic.output_channels[0].frequency = 1.0e9

# using the direct_rf_addon API together with the rfdc API to set the mixer frequency and nyquist zone.
qic.direct_rf_board.dac(0).nyquist = 1
qic.rfdc.dac(3, 3).mixer_frequency = 1.0e9

The `direct_rf` serves as a wrapper around the `direct_rf_addon` and `rfdc` modules. All methods applied to the channel will be reflected by the respecive addon and RFdc APIs.
Convenience methods exist to get the ADC or DAC resp. the addon channel for a given `direct_rf` channel

In [3]:
qic.output_channels[0].frequency = 500e6
qic.input_channels[1].frequency = 5.5e9

print(
    f"DAC frequency at channel 0: {qic.output_channels[0].rf_dac.mixer_frequency/1e9:.1f}GHz"
)
print(
    f"ADC frequency at channel 1: {qic.input_channels[1].rf_adc.mixer_frequency/1e9:.1f}GHz"
)

DAC frequency at channel 0: 0.5GHz
ADC frequency at channel 1: 5.5GHz
